In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd

In [3]:
dataTrain=pd.read_csv("../data/traindata.csv")
dataTest=pd.read_csv("../data/testdata.csv")
print(dataTrain.head())
print(dataTrain.shape)

   Unnamed: 0  round  venue   gf   ga   xg  xga  poss  formation    sh  ...  \
0           0     33      0  0.0  0.0  1.4  0.8  49.0          4  11.0  ...   
1           1     35      1  2.0  0.0  2.4  0.4  64.0          4  25.0  ...   
2           2     37      0  5.0  2.0  3.5  1.0  67.0          4  30.0  ...   
3           3      2      0  1.0  3.0  0.3  1.8  43.0          4   6.0  ...   
4           4      3      1  2.0  0.0  2.1  0.7  43.0          4  14.0  ...   

   sh_rolling  sot_rolling  dist_rolling  fk_rolling  pk_rolling  \
0   17.333333     6.333333     17.300000    0.333333         0.0   
1   12.000000     3.666667     16.566667    0.333333         0.0   
2   17.666667     6.333333     17.166667    1.000000         0.0   
3   22.000000     8.000000     17.500000    1.333333         0.0   
4   20.333333     8.333333     18.500000    1.666667         0.0   

  pkatt_rolling  home_code  target_away  target_draw  target_home  
0           0.0          0            0         

In [4]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1808 entries, 0 to 1807
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1808 non-null   int64  
 1   round          1808 non-null   int64  
 2   venue          1808 non-null   int64  
 3   gf             1808 non-null   float64
 4   ga             1808 non-null   float64
 5   xg             1808 non-null   float64
 6   xga            1808 non-null   float64
 7   poss           1808 non-null   float64
 8   formation      1808 non-null   int64  
 9   sh             1808 non-null   float64
 10  sot            1808 non-null   float64
 11  dist           1806 non-null   float64
 12  fk             1808 non-null   float64
 13  pk             1808 non-null   float64
 14  pkatt          1808 non-null   float64
 15  season         1808 non-null   object 
 16  venue_code     1808 non-null   int64  
 17  opp_code       1808 non-null   int64  
 18  hour    

In [5]:
dataTrain.head()

,Unnamed: 0,round,venue,gf,ga,xg,xga,poss,formation,sh,...,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,home_code,target_away,target_draw,target_home
0,0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,...,17.333333,6.333333,17.300000,0.333333,0.0,0.0,0,0,1,0
1,1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,...,12.000000,3.666667,16.566667,0.333333,0.0,0.0,0,0,0,1
2,2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,...,17.666667,6.333333,17.166667,1.000000,0.0,0.0,0,0,0,1
3,3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,...,22.000000,8.000000,17.500000,1.333333,0.0,0.0,0,1,0,0
4,4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,...,20.333333,8.333333,18.500000,1.666667,0.0,0.0,0,0,0,1


In [5]:
del dataTrain["Unnamed: 0"]
del dataTest["Unnamed: 0"]

In [6]:
del dataTrain["season"]
del dataTest["season"]

In [7]:
del dataTrain["target_draw"]
del dataTest["target_draw"]

In [22]:
# Data Training
train_data = dataTrain.iloc[:, :-2].values
train_labels = dataTrain.iloc[:, -2:].values

# Data Testing
test_data = dataTest.iloc[:, :-2].values
test_labels = dataTest.iloc[:, -2:].values

In [23]:
# Normalisasi Data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Pembentukan Data Sequence
sequence_length = 10  # Panjang sequence yang akan digunakan
num_features = train_data.shape[1]

In [24]:
def create_sequences(data, labels, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(labels[i+sequence_length])
    return np.array(X), np.array(y)

In [25]:
X_train, y_train = create_sequences(train_data, train_labels, sequence_length)
X_test, y_test = create_sequences(test_data, test_labels, sequence_length)

In [26]:
# Pembangunan Model LSTM
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, num_features)))
model.add(Dense(units=2, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Pelatihan Model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Sesuaikan dengan jumlah epoch dan ukuran batch yang optimal

Epoch 1/10
57/57 [==============================] - 3s 7ms/step - loss: 0.6690 - accuracy: 0.5195
Epoch 2/10
57/57 [==============================] - 0s 7ms/step - loss: 0.6491 - accuracy: 0.5997
Epoch 3/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6410 - accuracy: 0.6019
Epoch 4/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6384 - accuracy: 0.6203
Epoch 5/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6388 - accuracy: 0.6158
Epoch 6/10
57/57 [==============================] - 0s 7ms/step - loss: 0.6357 - accuracy: 0.6197
Epoch 7/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6348 - accuracy: 0.6192
Epoch 8/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6327 - accuracy: 0.6381
Epoch 9/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6306 - accuracy: 0.6342
Epoch 10/10
57/57 [==============================] - 0s 6ms/step - loss: 0.6357 - accuracy: 0.6297


In [28]:
# Evaluasi Model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

19/19 [==============================] - 1s 4ms/step - loss: 0.6469 - accuracy: 0.5960
Test Loss: 0.6469048857688904
Test Accuracy: 0.5959933400154114


In [16]:
dataTrain["target_home"].value_counts()

0    1112
1     696
Name: target_home, dtype: int64

In [17]:
dataTrain

,round,venue,gf,ga,xg,xga,poss,formation,sh,sot,...,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,home_code,target_away,target_home
0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,...,2.333333,17.333333,6.333333,17.300000,0.333333,0.0,0.0,0,0,0
1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,...,1.333333,12.000000,3.666667,16.566667,0.333333,0.0,0.0,0,0,1
2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,...,0.000000,17.666667,6.333333,17.166667,1.000000,0.0,0.0,0,0,1
3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,...,0.666667,22.000000,8.000000,17.500000,1.333333,0.0,0.0,0,1,0
4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,...,1.666667,20.333333,8.333333,18.500000,1.666667,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,24,0,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,...,1.333333,13.333333,4.000000,16.033333,0.666667,0.0,0.0,19,0,1
1804,28,1,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,...,1.333333,12.333333,4.333333,15.200000,0.666667,0.0,0.0,19,0,1
1805,29,0,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,...,1.333333,14.333333,4.666667,17.000000,0.666667,0.0,0.0,19,1,0
1806,30,0,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,...,1.000000,12.666667,4.000000,17.666667,0.666667,0.0,0.0,19,1,0


In [18]:
# Pengecekan null value pada data training
print(dataTrain.isnull().sum())

# Pengecekan null value pada data testing
print(dataTest.isnull().sum())

round            0
venue            0
gf               0
ga               0
xg               0
xga              0
poss             0
formation        0
sh               0
sot              0
dist             2
fk               0
pk               0
pkatt            0
venue_code       0
opp_code         0
hour             0
day_code         0
gf_rolling       0
ga_rolling       0
sh_rolling       0
sot_rolling      0
dist_rolling     0
fk_rolling       0
pk_rolling       0
pkatt_rolling    0
home_code        0
target_away      0
target_home      0
dtype: int64
round            0
venue            0
gf               0
ga               0
xg               0
xga              0
poss             0
formation        0
sh               0
sot              0
dist             0
fk               0
pk               0
pkatt            0
venue_code       0
opp_code         0
hour             0
day_code         0
gf_rolling       0
ga_rolling       0
sh_rolling       0
sot_rolling      0
dist_rolling     0

In [19]:
# Menghapus baris dengan null value pada data training
dataTrain = dataTrain.dropna()

# Menghapus baris dengan null value pada data testing
dataTest = dataTest.dropna()

In [20]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1806 entries, 0 to 1807
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          1806 non-null   int64  
 1   venue          1806 non-null   int64  
 2   gf             1806 non-null   float64
 3   ga             1806 non-null   float64
 4   xg             1806 non-null   float64
 5   xga            1806 non-null   float64
 6   poss           1806 non-null   float64
 7   formation      1806 non-null   int64  
 8   sh             1806 non-null   float64
 9   sot            1806 non-null   float64
 10  dist           1806 non-null   float64
 11  fk             1806 non-null   float64
 12  pk             1806 non-null   float64
 13  pkatt          1806 non-null   float64
 14  venue_code     1806 non-null   int64  
 15  opp_code       1806 non-null   int64  
 16  hour           1806 non-null   int64  
 17  day_code       1806 non-null   int64  
 18  gf_rolli

In [21]:
dataTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          609 non-null    int64  
 1   venue          609 non-null    int64  
 2   gf             609 non-null    float64
 3   ga             609 non-null    float64
 4   xg             609 non-null    float64
 5   xga            609 non-null    float64
 6   poss           609 non-null    float64
 7   formation      609 non-null    int64  
 8   sh             609 non-null    float64
 9   sot            609 non-null    float64
 10  dist           609 non-null    float64
 11  fk             609 non-null    float64
 12  pk             609 non-null    float64
 13  pkatt          609 non-null    float64
 14  venue_code     609 non-null    int64  
 15  opp_code       609 non-null    int64  
 16  hour           609 non-null    int64  
 17  day_code       609 non-null    int64  
 18  gf_rolling